In [17]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
# Normalize pixel values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [6]:
# Convert class labels to one-hot encoded vectors
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [7]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (32, 32, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

In [8]:
# Compile the model
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 128)        0

In [10]:
ckpt_model = 'cifar_10_cnn-weights.best.hdf5'
checkpoint = ModelCheckpoint(ckpt_model,
                            monitor = 'val_acc',
                            verbose = 1,
                            save_best_only = True,
                            mode = 'max')
early_stopping = EarlyStopping(patience = 10, 
                               monitor = 'val_loss', 
                               restore_best_weights = True)
callbacks_list = [checkpoint, early_stopping]

In [11]:
# Train the model
print('Starting training...')

history = model.fit(x_train,
                    y_train,
                    validation_data = (x_test, y_test),
                    epochs = 10,
                    batch_size = 32,
                    callbacks = callbacks_list,
                    verbose = 0)

Starting training...


In [12]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

313/313 [==============================] - 2s 7ms/step - loss: 0.8922 - accuracy: 0.7165
Test loss: 0.8922255635261536
Test accuracy: 0.7164999842643738


In [13]:
def plot_metric(history, metric, metric_name):
    train_metric = history.history[metric]
    test_metric = history.history['val_' + metric]
    x = list(range(1, len(test_metric) + 1))
    plt.plot(x, test_metric, color='red', label='test ' + metric_name)
    plt.plot(x, train_metric, label='training ' + metric_name)
    plt.xlabel('Epoch')
    plt.ylabel(metric_name.capitalize())
    plt.title(metric_name.capitalize() + ' vs. Epoch')
    plt.legend()

In [ ]:
plot_metric(history, 'loss', 'loss')

In [ ]:
plot_metric(history, 'accuracy', 'accuracy')